### Clear GPU

In [ ]:
from keras import backend as K

K.clear_session()

In [ ]:
import tensorflow as tf
from numba import cuda
import gc

def clear_memory():
    # Clear VRAM
    tf.keras.backend.clear_session()
    cuda.select_device(0)
    cuda.close()

    # Clear RAM
    gc.collect()

#This should clear the VRAM and RAM
clear_memory()

### Import the Datasets in my drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
training_set_path = '/content/drive/My Drive/[2024-2025] AN2DL/Homework 1'
folder_path = '/content/drive/My Drive/Datasets'

### Check GPU Existence and Status




In [ ]:
import tensorflow as tf

# List all GPUs TensorFlow detects
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow detected the following GPU(s):")
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Name: {details['device_name']}")

In [ ]:
#This is to check GPU-Status and Usage (works only for NVIDIA GPUs)
!nvidia-smi

physical_devices = tf.config.list_physical_devices('GPU')
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

### Check Tensorflow and Keras Version


In [ ]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

### Import all the libraries

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
import keras as tfk
from keras import layers as tfkl
from keras import regularizers

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import keras_cv

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

#Number of Classes in the Dataset
num_classes = 8

### Create a function to Load Data and load the datasets needed


In [ ]:
def load_data(path):
    # Load dataset from .npz file
    data = np.load(path)

    # Trim dataset to the first 11959 entries and discard the rest since they are garbage data
    train_dataset = data['images'][:11959].copy()  # Copy to ensure no reference to the original array
    test_dataset = data['labels'][:11959].copy()

    # Explicitly delete the original data to free up memory
    del data

    return train_dataset, test_dataset

In [ ]:
# Execute function and load data
(X, y) = load_data("training_set.npz")

print("Test set shape (images):", X.shape)
print("Test set shape (labels):", y.shape)

In [ ]:
# Execute function and load data
(X_test_aug, y_test_aug) = load_data("augmented_set.npz")

print("Test set shape (images):", X_test_aug.shape)
print("Test set shape (labels):", y_test_aug.shape)

#One-hot encoding
y_test_aug = tfk.utils.to_categorical(y_test_aug3, num_classes=8)

In [ ]:
# Execute function and load data
(X_test_aug3, y_test_aug3) = load_data("augmented_set3.npz")

print("Test set shape (images):", X_test_aug3.shape)
print("Test set shape (labels):", y_test_aug3.shape)

#One-hot encoding
y_test_aug3 = tfk.utils.to_categorical(y_test_aug3, num_classes=8)

In [ ]:
# Execute function and load data
(X_test_aug4, y_test_aug4) = load_data("augmented_set4.npz")

print("Test set shape (images):", X_test_aug4.shape)
print("Test set shape (labels):", y_test_aug4.shape)

#One-hot encoding
y_test_aug4 = tfk.utils.to_categorical(y_test_aug4, num_classes=8)

### Split the Dataset

In [ ]:
# Definiamo le proporzioni
train_size = 1.0 # 70% training
val_size = 1.0  # 15% validation
test_size = 0.30  # 15% test

# Calcoliamo gli indici di split
total_samples = len(X)
train_samples = int(total_samples * train_size)
val_samples = int(total_samples * val_size)

# Dividiamo i dati
X_train = X[:train_samples]
y_train = y[:train_samples]

X_val = X[:val_samples]
y_val = y[:val_samples]

X_test = X[train_samples + val_samples:]
y_test = y[train_samples + val_samples:]

# Liberiamo memoria
del X, y
gc.collect()

# Stampiamo le dimensioni per verifica
print(f"Training set shape: {X_train.shape} - {y_train.shape}")
print(f"Validation set shape: {X_val.shape} - {y_val.shape}")
print(f"Test set shape: {X_test.shape} - {y_test.shape}")

### One-hot Encoding

In [ ]:
# One-hot encoding usando keras
y_train = tfk.utils.to_categorical(y_train, num_classes=8)
y_val = tfk.utils.to_categorical(y_val, num_classes=8)
y_test = tfk.utils.to_categorical(y_test, num_classes=8)

# Stampiamo le dimensioni per verifica
print(f"Training labels shape after one-hot encoding: {y_train.shape}")
print(f"Validation labels shape after one-hot encoding: {y_val.shape}")
print(f"Test labels shape after one-hot encoding: {y_test.shape}")

### Compute Class Distribution in the Dataset

In [ ]:
# Calcola le class weights dalle y_train (prima di convertirle in one-hot)
from sklearn.utils.class_weight import compute_class_weight

# Ottieni le etichette originali dalle one-hot encoded
y_train_labels = np.argmax(y, axis=1)

# Calcola i class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

# Crea un dizionario di class weights
class_weights_dict = dict(enumerate(class_weights))

# Stampa le informazioni
print("Class distribution:")
for class_idx, count in enumerate(np.bincount(y_train_labels)):
    print(f"Class {class_idx}: {count} samples, weight = {class_weights_dict[class_idx]:.3f}")

# Visualizza anche in percentuale
total_samples = len(y_train_labels)
print("\nClass distribution (percentage):")
for class_idx, count in enumerate(np.bincount(y_train_labels)):
    percentage = (count / total_samples) * 100
    print(f"Class {class_idx}: {percentage:.1f}%")

### Define The First - Model

In [ ]:
# Initialise ConvNeXtXLarge model with pretrained weights, for transfer learning
convnext =  tf.keras.applications.ConvNeXtXLarge(
    include_top=False,             # Esclude il classificatore finale
    input_shape=(96, 96, 3),       # Dimensioni di input
    weights="imagenet",            # Pesi preaddestrati su ImageNet
    input_tensor=None,             # Tensor di input (lascia None per usare input_shape)
    pooling=None,                  # Nessun pooling; specifica 'avg' per GlobalAveragePooling
    classes=8,                     # Numero di classi (non usato se include_top=False)
    classifier_activation="softmax" # Attivazione del classificatore (non usato se include_top=False)
)


# Display a summary of the model architecture
convnext.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
# Specify 'to_file' argument with a path where you have write permissions
tfk.utils.plot_model(convnext, to_file='/tmp/model.png', expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Freeze all layers in ConvNeXtXLarge to use it solely as a feature extractor
convnext.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')


# We wanted to try this approach, so also adding an augmentation pipeline to the model insted of using pre augmented data
augmentation = keras_cv.layers.Augmenter([
    keras_cv.layers.AutoContrast(value_range=(0, 255)),
    keras_cv.layers.AugMix(severity=0.5, value_range=(0, 255)),
    keras_cv.layers.ChannelShuffle(),
    #keras_cv.layers.CutMix(),
    #keras_cv.layers.MixUp(),
    #keras_cv.layers.JitteredResize(target_size=(96, 96), scale_factor=(1.0, 1.2)),
    keras_cv.layers.RandAugment(magnitude=0.3, value_range=(0, 255)),
    keras_cv.layers.RandomAugmentationPipeline(
        layers=[
            keras_cv.layers.RandomChannelShift(value_range=(0, 255), factor=0.1),
            keras_cv.layers.RandomColorDegeneration(factor=0.3),
            keras_cv.layers.RandomCutout(height_factor=0.2, width_factor=0.2),
            keras_cv.layers.RandomHue(factor=0.1, value_range=(0, 255)),
            keras_cv.layers.RandomSaturation(factor=0.2),
            keras_cv.layers.RandomSharpness(factor=0.2, value_range=(0, 255)),
            keras_cv.layers.RandomShear(x_factor=0.15, y_factor=0.15),
            keras_cv.layers.Solarization(value_range=(0, 255))
        ],
        #augmentations_per_image=2
        augmentations_per_image=tf.squeeze(tf.random.categorical(
            tf.math.log([[0.1, 0.2, 0.4, 0.3]]),  # [0 layers: 10%, 1 layer: 20%, 2 layers: 40%, 3 layers: 30%]
            1
        ))
    ),
])


#Apply the augmentation pipeline
inputs = augmentation(inputs)


# Pass augmented inputs through the MobileNetV3Small feature extractor
x = convnext(inputs)

x = tfkl.GlobalAveragePooling2D(name='avg_pool')(x)

# Add a batch normalization layer
#x = tfkl.BatchNormalization(name='batch_norm')(x)

# Add a dense layer with 256 units and GELU activation
x = tfkl.Dense(256, activation='gelu', name='dense1')(x)


# Add layer normalizatiFinal_Project.ipynbon
x = tfkl.LayerNormalization(name='layer_norm1')(x)

# Add another dropout layer
x = tfkl.Dropout(0.4, name='dropout2')(x)

# Add a second dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense2')(x)

# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm2')(x)

# Add another dropout layer
x = tfkl.Dropout(0.3, name='dropout3')(x)

# Add a third dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense3')(x)
'''
# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm3')(x)

# Add another dropout layer
x = tfkl.Dropout(0.3, name='dropout4')(x)
'''
# Add final Dense layer for classification with softmax activation
outputs = tfkl.Dense(8, activation='softmax', name='output')(x)

# Define the complete model linking input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with categorical cross-entropy loss and Lion optimiser
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer= tfk.optimizers.Adam(), metrics=['accuracy'])

# Display a summary of the model architecture
tl_model.summary(expand_nested=True)

### Train First - Model


In [ ]:
# Train the model
tl_history = tl_model.fit(
    x=X_train,
    y=y_train,
    batch_size=30,
    epochs=20,
    class_weight=class_weights_dict,
    validation_data=(X_val , y_val),
    callbacks=[tfk.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        tfk.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        ]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
#model_filename = 'Blood_Cells_MobileNetV3S_' + str(final_val_accuracy) + '.keras'
#tl_model.save(model_filename)

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'Blood_Cells_MobileNetV3S_' + str(final_val_accuracy) + '.keras.weights.h5'
tl_model.save_weights(model_filename)

### Test the First - Model


In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = tl_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

del tl_model
clear_memory()

### First Fine - Tuning


In [ ]:
# Initialise MobileNetV3Small model with pretrained weights, for transfer learning
mobilenetf = tf.keras.applications.ConvNeXtXLarge(
    include_top=False,
    input_shape=(96, 96, 3),
    weights=None,
    input_tensor=None,
    pooling=False,
    classes=8,
    classifier_activation="softmax"
)

# Freeze all layers in MobileNetV3Small to use it solely as a feature extractor
mobilenetf.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')


# Definisci il pipeline completo di augmentazione
augmentation = keras_cv.layers.Augmenter([
    keras_cv.layers.AutoContrast(value_range=(0, 255)),
    keras_cv.layers.AugMix(severity=0.5, value_range=(0, 255)),
    keras_cv.layers.ChannelShuffle(),
    #keras_cv.layers.CutMix(),
    #keras_cv.layers.MixUp(),
    #keras_cv.layers.JitteredResize(target_size=(96, 96), scale_factor=(1.0, 1.2)),
    keras_cv.layers.RandAugment(magnitude=0.3, value_range=(0, 255)),
    keras_cv.layers.RandomAugmentationPipeline(
        layers=[
            keras_cv.layers.RandomChannelShift(value_range=(0, 255), factor=0.1),
            keras_cv.layers.RandomColorDegeneration(factor=0.3),
            keras_cv.layers.RandomCutout(height_factor=0.2, width_factor=0.2),
            keras_cv.layers.RandomHue(factor=0.1, value_range=(0, 255)),
            keras_cv.layers.RandomSaturation(factor=0.2),
            keras_cv.layers.RandomSharpness(factor=0.2, value_range=(0, 255)),
            keras_cv.layers.RandomShear(x_factor=0.15, y_factor=0.15),
            keras_cv.layers.Solarization(value_range=(0, 255))
        ],
        #augmentations_per_image=2
        augmentations_per_image=tf.squeeze(tf.random.categorical(
            tf.math.log([[0.1, 0.2, 0.4, 0.3]]),  # [0 layers: 10%, 1 layer: 20%, 2 layers: 40%, 3 layers: 30%]
            1
        ))
    ),
])


#Apply the augmentation pipeline
inputs = augmentation(inputs)


# Pass augmented inputs through the MobileNetV3Small feature extractor
x = mobilenetf(inputs)

x = tfkl.GlobalAveragePooling2D(name='avg_pool')(x)

# Add a batch normalization layer
#x = tfkl.BatchNormalization(name='batch_norm')(x)

# Add a dense layer with 256 units and GELU activation
x = tfkl.Dense(256, activation='gelu', name='dense1')(x)


# Add layer normalizatiFinal_Project.ipynbon
x = tfkl.LayerNormalization(name='layer_norm1')(x)

# Add another dropout layer
x = tfkl.Dropout(0.4, name='dropout2')(x)

# Add a second dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense2')(x)

# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm2')(x)

# Add another dropout layer
x = tfkl.Dropout(0.3, name='dropout3')(x)

# Add a third dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense3')(x)
'''
# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm3')(x)

# Add another dropout layer
x = tfkl.Dropout(0.3, name='dropout4')(x)
'''
# Add final Dense layer for classification with softmax activation
outputs = tfkl.Dense(8, activation='softmax', name='output')(x)

# Define the complete model linking input and output
ft_model  = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with categorical cross-entropy loss and Adam optimiser
ft_model .compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

# Load the saved weights
model_filename = 'Blood_Cells_MobileNetV3S_80.15.keras.weights.h5'  # replace <final_val_accuracy> with the actual accuracy
ft_model.load_weights(model_filename)


In [ ]:
# Re-load the model after transfer learning
#ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_85.5.keras')
ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_'+ str(final_val_accuracy) + '.keras')

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Set the MobileNetV3Small model layers as trainable
ft_model.get_layer('convnext_xlarge').trainable = True

# Set all MobileNetV3Small layers as non-trainable
for layer in ft_model.get_layer('convnext_xlarge').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

In [ ]:
# Set the number of layers to freeze
N = 124

# Set the first N layers as non-trainable
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
    print(i, layer.name, layer.trainable)

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Compile the model with categorical cross-entropy loss and Adam optimiser
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = X,          # Corretto: usa training set
    y = y,          # Corretto: usa training set
    batch_size = 32,
    epochs = 30,          # Aumentato per dare più tempo al training
    validation_data = (X, y),
    callbacks = [
        tfk.callbacks.EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=7,    # Aumentato per evitare stop prematuro
            restore_best_weights=True
        ),
        tfk.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,    # Aumentato ma mantenuto minore dell'EarlyStopping
            min_lr=1e-7,
            verbose=1,
            mode='min',
            min_delta=1e-4,
            cooldown=1
        )
    ]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'Blood_Cells_MobileNetV3S_'+str(final_val_accuracy)+'.keras'
ft_model.save(model_filename)

### Second Fine - Tuning

In [ ]:
# Re-load the model after transfer learning
#ft_model = tfk.models.load_model('/content/Blood_Cells_MobileNetV3S_75.08.keras')
ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_'+ str(final_val_accuracy) + '.keras')

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Set the MobileNetV3Small model layers as trainable
ft_model.get_layer('convnext_xlarge').trainable = True

# Set all MobileNetV3Small layers as non-trainable
for layer in ft_model.get_layer('convnext_xlarge').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

In [ ]:
# Set the number of layers to freeze
N = 100

# Set the first N layers as non-trainable
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(ft_model.get_layer('convnext_xlarge').layers):
    print(i, layer.name, layer.trainable)

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Compile the model with categorical cross-entropy loss and Adam optimiser
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
# Enable mixed precision
tfk.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 64,
    epochs = 5,
    validation_data = (X_test, y_test),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=False),
                 tfk.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,        # Riduzione graduale per un modello grande
                    patience=1,        # ~25-30% delle epoche totali
                    min_lr=1e-7,      # Considerando Adam come optimizer
                    verbose=1,
                    mode='min',
                    min_delta=1e-4,   # Basato sulla scala delle tue loss
                    cooldown=1        # Breve periodo di stabilizzazione
)]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'Blood_Cells_MobileNetV3S_'+str(final_val_accuracy)+'.keras'
ft_model.save(model_filename)

### Testing Second Fine - Tuning


In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = ft_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = ft_model.predict(X_test_aug)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test_aug.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = ft_model.predict(X_test_aug2)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test_aug2.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = ft_model.predict(X_test_aug3)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test_aug3.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = ft_model.predict(X_test_aug4)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test_aug3.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

### Submit Section


In [ ]:
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('Blood_Cells_MobileNetV3S_99.12.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
# The original path was incorrect. Using f-string to format correctly.
!zip {filename} model.py Blood_Cells_MobileNetV3S_99.12.keras

### Copy the submission file into the drive


In [ ]:
!cp submission_241116_200916.zip /content/drive/MyDrive/